In [2]:
import mosaik
import mosaik.util

# SIM_Config defines which simulators we can run.
SIM_CONFIG = {
    'PVSim': {
        'python': 'mosaik_pv:PVSim',
    },
    'BatterySim': {
        'python': 'mosaik_battery:BatterySim',
    },
    'DemandSim': {
        'python': 'mosaik_demand:DemandSim',
    },
    'GridSim': {
        'python': 'mosaik_grid:GridSim',
    },
    'ControlSim': {
        'python': 'mosaik_control:ControlSim',
    },
    'CollectorSim': {
        'python': 'collector:Collector',
    },
    'ThermalHouseSim':{
        'python': 'mosaik_thermalHouse:ThermalHouseSim',
    },
    'ControlTempSim': {
        'python': 'mosaik_control_temp:ControlTempSim',
    }
}

In [3]:
def init_entities(world):
    sim_dict = {}
    entity_dict = {}
    ## Simulators

    ## Photovoltaic Producer
    # Params:
    #   rated_capacity: Rated (maximal) power [kW]
    #   seriesname: Name of series in the data store
    # Output:
    #   P: Current production [kW]
    #   Pav: Current maximally available production [kW]
    #   Pmax: Current power limit [kW]
    # Input:
    #   Plimit: Current power limit [kW]
    # Desc:
    #   Read PV production from a CSV file (=> Pav)
    #   P = min(Pav, Plimit)

    pv_sim = world.start(
            'PVSim',
            eid_prefix='pv_',
            step_size=1)
    pv_entity_1 = pv_sim.PVSim(
            rated_capacity=7,
            series_name='/PV715_20180510')
    sim_dict['pv'] = pv_sim
    entity_dict['pv1'] = pv_entity_1
    
    pv_sim2 = world.start(
            'PVSim',
            eid_prefix='pv2_',
            step_size=1)
    pv_entity_2 = pv_sim2.PVSim(
            rated_capacity=7,
            series_name='/PV715_20180511')
    sim_dict['pv2'] = pv_sim2
    entity_dict['pv2_1'] = pv_entity_2
    
    pv_entity_3 = pv_sim2.PVSim(
            rated_capacity=1,
            series_name='/PV715_20180512')
    entity_dict['pv2_2'] = pv_entity_3
    
    
    ## Battery
    # Params:
    #   rated_charge_capacity: Maximum charge power (>0) [kW]
    #   rated_discharge_capacity: Maximum discharge power (>0) [kW]
    #   rated_capacity: Maximal state of charge (>0) [kWs]
    #   initial_charge_rel: Battery fill % at start (in [0,1]) [r.u.]
    #   charge_change_rate: Time-delay factor (in [0,1], 1 => instant response)
    #   dt: Conversion factor for one time step (default: 1.0/(60*60) = 1sec/time step)
    # Output:
    #   P: Current production/consumption [kW]
    #   SoC: Current state-of-charge (in [0, SoCmax]) [kWs]
    # Input:
    #   Pset: Current power target [kW]
    # Desc:
    #   P>0 => the battery is outputting power into the grid (discharging)
    #   Pset = Pset bounded to [-Pmax, Pmax]
    #   P(t) = kappa*Pset + (1-kappa)*P(t-1) bounded to [-SoC, SoCmax-SoC]
    #   SoC(t) = SoC(t-1) + P(t) bounded to [0, SoCmax]

    batt_sim = world.start(
            'BatterySim',
            eid_prefix='batt_',
            step_size=1)
    batt_entity_1 = batt_sim.BatterySim(
            rated_capacity=10,
            rated_discharge_capacity=30,
            rated_charge_capacity=30,
            initial_charge_rel=0.5,
            charge_change_rate=0.03,
    #        dt=1.0/(60*60)
            )
    sim_dict['batt'] = batt_sim
    entity_dict['batt1'] = batt_entity_1
    
    ## Demand
    # Params:
    #   rated_capacity: Maximal power draw (indicative) [kW]
    # Output:
    #   P: Current power draw [kW]
    # Desc:
    #   Read demand time series from a CSV file (=> Prel in [0, 1])
    #   P = Pmax * Prel

    demand_sim = world.start(
            'DemandSim',
            eid_prefix='demand_',
            step_size=1)
    demand_entity_1 = demand_sim.DemandSim(rated_capacity=4, seriesname='/flexhouse_20180218')
    sim_dict['demand'] = demand_sim
    entity_dict['demand1'] = demand_entity_1
    

    ## Grid model
    # Params:
    #   droop: Relationship between power draw and voltage change [V/kW] (usually <0)
    #   V0: Base voltage [V]
    # Output:
    #   V: Current voltage at bus [V]
    #   Pgrid: Current power draw from the grid [kW]
    # Input:
    #   P: Current power draw from units (Many connections) [kW]
    # Desc:
    #   Pgrid = - (P_1 + ... + P_N)
    #   V = V0 + droop * Pgrid

    grid_sim = world.start(
            'GridSim',
            eid_prefix='grid_',
            step_size=1)
    grid_entity_1 = grid_sim.GridSim(V0=240, droop=0.1)
    
    sim_dict['grid'] = grid_sim
    entity_dict['grid1'] = grid_entity_1
    
    ## Controller
    # Params:
    #   kappa: Time-delay compensation factor (usually in [0, 1], 1 => instant response) [a.u.]
    # Input:
    #   Pgrid: Current power draw [kW]
    #   SoC: Battery state-of-charge [kWs]
    # Output:
    #   Pset: Battery power setpoint [kW]
    # Desc:
    #   Psett(t) = kappa * Pgrid + (1 - kappa) * Psett(t-1)

    control_sim = world.start(
            'ControlSim',
            eid_prefix='control_',
            step_size=1)
    control_entity_1 = control_sim.ControlSim(setpoint_change_rate=0.50)
    
    sim_dict['control'] = control_sim
    entity_dict['control1'] = control_entity_1
    
    ## Collector
    # Params:
    #   Storefilename: String indicating the storage file to use
    # Input:
    #   GENERIC: Anything can be connected and will be logged
    # Desc:
    #   During simulation, save system state
    #   After simulation, write collected data to disk into the 'datastore' file with the given name.

    collector_sim = world.start(
            'CollectorSim',
            step_size=60,
            save_h5=True,
            h5_storename='datastore',
            h5_panelname='Monitor_SimpleCase_Centralized',
            print_results=False)
    collector_entity = collector_sim.Collector()
    
    sim_dict['collector'] = collector_sim
    entity_dict['collector'] = collector_entity

    ## Thermal House model
    # Params:
    #   ambient_temp: Ambient temperature considered constant
    #   house_insulation_coef: House insulation coefficient [1/h]
    #   solar_heating_coef: Solar heating coefficient [ºC/h](in [0,1])
    #   heating_coef: Heating coefficient [ºC/h]
    #   t_init: Initial house temperature [ºC]
    #   dt: Conversion factor for one time step (default: 1.0/(60*60) = 1sec/time step) ?????????????????
    # Output:
    #   T: Current inside temperature [kW]
    # Input:
    #   zs: Current solar heating amount [0,1]
    #   x: Heater setting [0,1]
    
    thermHouse_sim = world.start(
            'ThermalHouseSim',
            eid_prefix='thermHouse_',
            step_size=1)
    thermHouse_entity_1 = thermHouse_sim.ThermalHouseSim(
            ambient_temp=12,
            house_insulation_coef=0.2/3600,
            solar_heating_coef=6.0/3600,
            heating_coef=12.0/3600,
            t_init=16,
    #        dt=1.0/(60*60)
            )
    sim_dict['thermHouse'] = thermHouse_sim
    entity_dict['thermHouse1'] = thermHouse_entity_1
    
    ## Temp Controller
    # Params:
    #   kappa: Time-delay compensation factor (usually in [0, 1], 1 => instant response) [a.u.]
    #   Tsp: Temperature setpoint [ºC]
    #   kp: Controller gain
    # Input:
    #   T: Current house temperature [ºC]
    # Output:
    #   x: Heater setting [0,1]

    control_temp_sim = world.start(
            'ControlTempSim',
            eid_prefix='controlT_',
            step_size=1)
    control_temp_entity_1 = control_temp_sim.ControlTempSim()
    
    sim_dict['control_temp'] = control_temp_sim
    entity_dict['control_temp1'] = control_temp_entity_1
    
    return sim_dict, entity_dict

In [4]:
END = 24*60*60-1 # 24 hours, 1 MosaikTime = 1 second

# Init world
world = mosaik.World(SIM_CONFIG)

# Add entities
sim_dict, entity_dict = init_entities(world)

###
#  Connect components
###

# Connect units to grid busbar
world.connect(entity_dict['demand1'], entity_dict['grid1'], ('P', 'P'))
world.connect(entity_dict['pv1'], entity_dict['grid1'], ('P', 'P'))
world.connect(entity_dict['pv2_1'], entity_dict['grid1'], ('P', 'P'))
world.connect(entity_dict['pv2_2'], entity_dict['grid1'], ('P', 'P'))
world.connect(entity_dict['batt1'], entity_dict['grid1'], ('P', 'P'))

# PV 2 House
world.connect(entity_dict['pv2_2'], entity_dict['thermHouse1'], ('P', 'zs'))

# Connect controller
world.connect(entity_dict['grid1'], entity_dict['control1'], ('Pgrid', 'Pgrid'))
world.connect(entity_dict['control1'], entity_dict['batt1'], ('Pset', 'Pset'), time_shifted=True, initial={'Pset': 0.0})
world.connect(entity_dict['batt1'], entity_dict['control1'], ('relSoC', 'relSoC'))
world.connect(entity_dict['control_temp1'], entity_dict['thermHouse1'], ('x', 'x'), time_shifted=True, initial={'x': 0.0})
world.connect(entity_dict['thermHouse1'], entity_dict['control_temp1'], ('T', 'T'))

# Connect to Collector
world.connect(entity_dict['batt1'], entity_dict['collector'], ('P', 'BattP'))
world.connect(entity_dict['batt1'], entity_dict['collector'], ('SoC', 'BattSoC'))
world.connect(entity_dict['demand1'], entity_dict['collector'], ('P', 'DemP'))
world.connect(entity_dict['pv1'], entity_dict['collector'], ('P', 'SolarP'))
world.connect(entity_dict['pv2_1'], entity_dict['collector'], ('P', 'SolarP2'))
world.connect(entity_dict['pv2_2'], entity_dict['collector'], ('P', 'SolarP3'))
world.connect(entity_dict['grid1'], entity_dict['collector'], ('Pgrid', 'GridP'))
world.connect(entity_dict['thermHouse1'], entity_dict['collector'], ('T', 'T'))
world.connect(entity_dict['thermHouse1'], entity_dict['collector'], ('x', 'radiator'))

# Run simulation
world.run(END)


Starting "PVSim" as "PVSim-0" ...
{'api_version': '2.3', 'models': {'PVSim': {'public': True, 'params': ['series_name', 'rated_capacity'], 'attrs': ['P', 'Pav', 'Pmax']}}}
Starting "PVSim" as "PVSim-1" ...
{'api_version': '2.3', 'models': {'PVSim': {'public': True, 'params': ['series_name', 'rated_capacity'], 'attrs': ['P', 'Pav', 'Pmax']}}}
Starting "BatterySim" as "BatterySim-0" ...
{'api_version': '2.3', 'models': {'BatterySim': {'public': True, 'params': ['rated_capacity', 'rated_charge_capacity', 'rated_discharge_capacity', 'roundtrip_efficiency', 'initial_charge_rel', 'charge_change_rate'], 'attrs': ['P', 'Pset', 'SoC', 'relSoC']}}}
Starting "DemandSim" as "DemandSim-0" ...
{'api_version': '2.3', 'models': {'DemandSim': {'public': True, 'params': ['seriesname', 'rated_capacity'], 'attrs': ['P']}}}
Starting "GridSim" as "GridSim-0" ...
{'api_version': '2.3', 'models': {'GridSim': {'public': True, 'params': ['V0', 'droop'], 'attrs': ['P', 'Pgrid', 'V']}}}
Starting "ControlSim" as "

/opt/conda/lib/python3.6/site-packages/pandas/core/panel.py:277: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  d = cls._homogenize_dict(cls, data, intersect=intersect, dtype=dtype)
/home/jovyan/work/simple_study_case/collector.py:70: FutureWarning: 
Panel is deprecated and will be removed in a future version.
The recommended way to represent these types of 3-dimensional data are with a MultiIndex on a DataFrame, via the Panel.to_frame() method
Alternatively, you can use the xarray package http://xarray.pydata.org/en/stable/.
Pandas provides a `.to_xarray()` method to help automate this conversion.

  panel = pd.Panel.from_dict({k: pd.DataFrame(v, index=self.time_list) for k,v in self.data.items()})
